## XGBoost_11Feature_LeaveOneGroupOut

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd  
import seaborn as sns 
from sklearn.datasets import load_boston
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jul-2021/test.csv')
print('train shape:',train.shape)
print('test shape:',test.shape)

In [ ]:
# found that one of the test data is in the train set and removed it
# Dropping the last row which is 2011-01-01 00:00:00
train=train.loc[~(train['date_time']=='2011-01-01 00:00:00')].reset_index(drop=True)

In [ ]:
print('train shape:',train.shape)

In [ ]:
all_data = pd.concat([train, test])
# convert to datatime format
all_data['date_time'] = pd.to_datetime(all_data['date_time'])
all_data.head()

## EDA
- The distribution of deg_C shows peaks between 20 to 30 deg.
- There is a dip in relative humidity at 40% and there are two peaks at 30% and 45% approx.
- The absolute humidity value shows peaks at 0.25g/m3(i have assumed it to be g/m3.Data info did not explicitly mention any units).
- The distribution of sensor_1,2,3 & 5 appears to be left skewed whereas sensor-4 is normal with outliers at 500.

In [ ]:
fig,ax=plt.subplots(4,2,figsize=(20,15))
for i,col in enumerate(train.columns[1:9]):
    ax[i%4][i//4].hist(train[col],bins=40,color='darkblue',label=f'{col}')
    ax[i%4][i//4].set_title(f'Distribution of {col}',fontsize=15)
    ax[i%4][i//4].set_xlabel(f'{col}')
    ax[i%4][i//4].set_ylabel('Dist')
    plt.subplots_adjust(hspace=0.45)

In [ ]:
fig,ax=plt.subplots(3,1,figsize=(8,10))
for i,col in enumerate(train.columns[9:12]):
    ax[i%3].hist(train[col],bins=40,color='darkblue',label=f'{col}')
    ax[i%3].set_title(f'Distribution of {col}',fontsize=15)
    ax[i%3].set_xlabel(f'{col}')
    ax[i%3].set_ylabel('Dist')
    plt.subplots_adjust(hspace=0.45)

### 分析每天的資訊
Looking at the day wise trend,we see that there has been sudden peak and dips for certain days over the month.While the temperatures have been above 20 deg after late may, there is a dip in temperature less than 15 dec after Nov but there is a sudden increase in mid december.

In [ ]:
# 字串轉換日期格式
train['date_time']=pd.to_datetime(train['date_time'],format='%Y-%m-%d %H:%M:%S')
test['date_time']=pd.to_datetime(test['date_time'],format='%Y-%m-%d %H:%M:%S')
# Following code is inspired from - https://www.kaggle.com/nroman/eda-for-ashrae
fig,ax=plt.subplots(1,1,figsize=(12,6))
train[['date_time','deg_C']].set_index('date_time').resample('D').mean()['deg_C'].plot(ax=ax,label='by hour(train)',alpha=1,color='blue').set_ylabel('deg C',fontsize=10)
ax.set_title('Trend of Mean deg_C by Day',fontsize=12)
ax.set_xlabel('')

For the test set, if we try to compare between March month of train, we could see the temperatures have started from approx 3 deg and increased above 15 deg.

In [ ]:
fig,ax=plt.subplots(1,1,figsize=(12,6))
test[['date_time','deg_C']].set_index('date_time').resample('D').mean()['deg_C'].plot(ax=ax,label='by hour(train)',alpha=1,color='blue').set_ylabel('deg C',fontsize=10)
ax.set_title('Trend of Mean deg_C by Day',fontsize=12)
ax.set_xlabel('')

## Data preprocessing
進一步處理之前先確認是否有缺失值：

In [ ]:
all_data.isnull().sum()

In [ ]:
# The months will be used for folds split
months = all_data["date_time"].dt.month[:len(train)]
## New idea
all_data["hour"] = all_data["date_time"].dt.hour
all_data["working_hours"] =  all_data["hour"].isin(np.arange(8, 21, 1)).astype("int")
all_data["is_weekend"] = (all_data["date_time"].dt.dayofweek >= 5).astype("int")
all_data.drop(columns = 'hour', inplace = True)
# convert datetime to timestamp(s)
all_data['time'] = all_data['date_time'].astype(np.int64)//10**9
all_data.drop(columns = 'date_time', inplace = True)

In [ ]:
X=all_data[:len(train)].drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
y=all_data[:len(train)][['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides']]
y_log=np.log1p(y)
X_test=all_data[len(train):].drop(columns = ['target_carbon_monoxide', 'target_benzene', 'target_nitrogen_oxides'])
print('X_train shape:', X.shape)
print('y_train shape:', y.shape)
print('X_test shape:', X_test.shape)

### Standardization 平均&變異數標準化
將所有特徵標準化，也就是高斯分佈。使得數據的平均值為0，方差為1。
適合的使用時機於當有些特徵的方差過大時，使用標準化能夠有效地讓模型快速收斂。

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

In [ ]:
fig,ax=plt.subplots(4,2,figsize=(20,15))
for i in range(8):
    col=train.columns[1:9][i]
    ax[i%4][i//4].hist(X_test_scaled[:,i],bins=40,color='darkblue',label=f'{col}')
    ax[i%4][i//4].set_title(f'Distribution of {col}',fontsize=15)
    ax[i%4][i//4].set_xlabel(f'{col}')
    ax[i%4][i//4].set_ylabel('Dist')
    plt.subplots_adjust(hspace=0.45)

In [ ]:
fig,ax=plt.subplots(3,1,figsize=(8,10))
for i in range(3):
    col=train.columns[9:12][i]
    ax[i%3].hist(y_log[:][col],bins=40,color='darkblue',label=f'{col}')
    ax[i%3].set_title(f'Distribution of {col}',fontsize=15)
    ax[i%3].set_xlabel(f'{col}')
    ax[i%3].set_ylabel('Dist')
    plt.subplots_adjust(hspace=0.45)

## Train Model

In [ ]:
# load submission
preds = pd.read_csv('../input/tabular-playground-series-jul-2021/sample_submission.csv')

## StratifiedKFold
Public Score: 0.22277

In [ ]:
# %%time
# from sklearn.metrics import mean_squared_log_error
# from sklearn.model_selection import StratifiedKFold
# import xgboost as xgb

# all_fi = []
# splits = 10

# for i, target in enumerate(y_log.columns):
#     print(f"\nTraining for {target}...")
#     skf = StratifiedKFold(n_splits=splits, shuffle = True, random_state = 42)
#     oof_preds = np.zeros((X_scaled.shape[0],))
#     model_preds = 0
#     model_fi = 0
#     for num, (train_idx, valid_idx) in enumerate(skf.split(X_scaled, months)):
#         X_train, X_valid = X_scaled[[train_idx]], X_scaled[[valid_idx]]
#         y_train, y_valid = y_log.loc[train_idx, target], y_log.loc[valid_idx, target]
#         model = xgb.XGBRegressor(max_depth=6,
#                        n_estimators=10000,
#                        random_state=42)
#         model.fit(X_train, y_train,
#                   eval_set=[(X_valid, y_valid)],
#                   verbose=False,early_stopping_rounds=500,)
#         model_preds += np.expm1(model.predict(X_test_scaled)) / splits
#         model_fi += model.feature_importances_
#         oof_preds[valid_idx] = np.expm1(model.predict(X_valid))
#         print(f"Fold {num} RMSLE: {np.sqrt(mean_squared_log_error(np.expm1(y_valid), oof_preds[valid_idx]))}")
#     print(f"\nOverall RMSLE: {np.sqrt(mean_squared_log_error(np.expm1(y_log[target]), oof_preds))}")    
#     preds[target] = model_preds
#     all_fi.append(model_fi)

## LeaveOneGroupOut
Public Score: 0.21989

train Score:0.198500193

In [ ]:
%%time
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import LeaveOneGroupOut
import xgboost as xgb

all_fi = []
splits = 10
target_names=y_log.columns

for i, target in enumerate(target_names):
    print(f"\nTraining for {target}...")
    logo = LeaveOneGroupOut()
    oof_preds = np.zeros((X_scaled.shape[0],))
    model_preds = 0
    model_fi = 0
    for num, (train_idx, valid_idx) in enumerate(logo.split(X_scaled, y_log, months)):
        X_train, X_valid = X_scaled[[train_idx]], X_scaled[[valid_idx]]
        y_train, y_valid = y_log.loc[train_idx, target], y_log.loc[valid_idx, target]
        model = xgb.XGBRegressor(max_depth=6,
                       n_estimators=10000)
        model.fit(X_train, y_train,
                  eval_set=[(X_valid, y_valid)],
                  verbose=False,early_stopping_rounds=500,)
        model_preds += np.expm1(model.predict(X_test_scaled)) / splits
        model_fi += model.feature_importances_
        oof_preds[valid_idx] = np.expm1(model.predict(X_valid))
        print(f"Fold {num} RMSLE: {np.sqrt(mean_squared_log_error(np.expm1(y_valid), oof_preds[valid_idx]))}")
    print(f"\nOverall RMSLE: {np.sqrt(mean_squared_log_error(np.expm1(y_log[target]), oof_preds))}")    
    preds[target] = model_preds
    all_fi.append(dict(zip(X.columns, model_fi)))

## Feature importances
Thanks to: @Maxim Kazantsev https://www.kaggle.com/maximkazantsev/tps-07-21-eda-catboost

In [ ]:
# Creating feature list from feature importance dictionaries
feature_list = set()
for i in np.arange(len(all_fi)):
    feature_list = set.union(feature_list, set(all_fi[i].keys()))
print(f"There are {len(feature_list)} unique features used for training: {feature_list}")

In [ ]:
# Combining feature importances of different models into one dataframe
df = pd.DataFrame(columns=["Feature"])
df["Feature"] = list(feature_list)
for i in np.arange(len(all_fi)):
    for key in all_fi[i].keys():
        df.loc[df["Feature"] == key, "Importance_" + str(i+1)] = all_fi[i][key] / 1000
df.fillna(0, inplace=True)
df.sort_values("Importance_1", axis=0, ascending=False, inplace=True)

In [ ]:
x = np.arange(0, len(df["Feature"]))
height = 0.3

fig, ax = plt.subplots(figsize=(12, 9))
bars1 = ax.barh(x-height, df["Importance_1"], height=height,
                color="cornflowerblue",
                edgecolor="black",
                label=target_names[0])
bars2 = ax.barh(x, df["Importance_2"], height=height,
                color="palevioletred",
                edgecolor="black",
                label=target_names[1])
bars3 = ax.barh(x+height, df["Importance_3"], height=height,
                color="mediumseagreen",
                edgecolor="black",
                label=target_names[2])
ax.set_title("Feature importances", fontsize=20, pad=5)
ax.set_ylabel("Feature names", fontsize=15, labelpad=5)
ax.set_xlabel("Feature importance", fontsize=15, labelpad=5)
ax.set_yticks(x)
ax.set_yticklabels(df["Feature"], fontsize=12)
ax.tick_params(axis="x", labelsize=12)
ax.grid(axis="x")
ax.legend(fontsize=13, loc="lower right")
plt.margins(0.04, 0.01)
plt.gca().invert_yaxis()

## Save Predict File

In [ ]:
preds.head()

In [ ]:
preds.to_csv('submission.csv', index=False)